In [12]:
import numpy as np

from typing import Callable, List, Tuple

N = 200
eps = 10**9

$$\int^1_0\frac {sin(\pi x^5)dx} {x^5 (1 - x)}$$
У интеграла две особенности
$$ \lim_{x \rightarrow 0} \frac {\sin(\pi x^5)} {x^5 (1 - x)} = 
    \pi \cdot \lim_{x \rightarrow 0} \frac {\sin(\pi x^5)} {\pi x^5} \cdot \frac {1}{ 1 - x } = \pi $$

$$ \lim_{x \rightarrow 1} \frac {\sin(\pi x^5)} {x^5 (1 - x)} = (0/0) =
    \pi \cdot \lim_{x \rightarrow 1} \frac {5x^4 \cdot \cos(\pi x^5)} {5x^4 - 6x^5} = 
    \pi \cdot \lim_{x \rightarrow 1} \frac {5\cos(\pi x ^ 5)} { 5 - 6x } = 5 \pi $$
    
Следовательно можно доопределить по непрерывности функцию в точке ноль и значение интеграла Римана не 
поменяется


In [13]:
def f(x : float) -> float:
    if (np.abs(x) < eps) :
        return np.pi 
    if (np.abs(x - 1) < eps):
        return 5 * np.pi
    return np.sin(np.pi * x ** 5) / ( x ** 5 * (1 - x))

$$\int_0^\infty e ^ { -\sqrt{x} + \sin(x/10) } dx = 
\int_0^1 \frac {e ^ { -\sqrt {\frac {t} {1 - t}} + \sin( \frac {t} {10(1 - t)})} } {(1 - t)^2}dt
$$

In [14]:
def g(t):
    x = t / (1 - t)
    return np.e ** (-np.sqrt(x) + np.sin(x/10))

In [15]:

def h(x : float) -> float: 
    return np.sin(x) 


In [16]:
#simpson formula
def integration(f : Callable[[float], float], interval : Tuple[float, float], n : int) -> float:
    a, b = interval
    h = (b - a) / n
    res = 0
    for i in range(1, n, 2):
        res += f(a + (i - 1) * h) + 4 * f(a + i * h) + f(a + (i + 1) * h)
    res *= h / 3
    return res 

In [41]:
# Gauss formula for n = 6

gauss_order = 6 

#some static data
t = [-0.93246951, 0.93246951, -0.66120939, 0.66120939, -0.23861919, 0.23861919]
A = [0.17132450, 0.36076158, 0.46791394, 0.46791394, 0.36076158, 0.17132450]

def gauss_integration(f: Callable[[float], float], interval : Tuple[float, float], n : int):
    a, b = interval
    h = (b - a) / (n - 1)
    half_sum = (b + a) / 2
    half_difference = (b - a) / 2
    x = [half_sum + half_difference * t[i] for i in range(n)]
    c = [((b - a) / 2) * A[i] * f(x[i]) for i in range(n)]
    print(x)

    return (sum(c))

gauss_integration(h, (0, np.pi), gauss_order)


[0.10607664563865948, 0.5321710457406024, 1.195974179640123, 1.94561847394967, 2.6094216078491907, 3.0355160079511334]


2.0000000237618814

Оценка порядка на тестовом интеграле:\
Теретическая оценка порядка  $O(h^4)$
Как получить оценку на практике?
У нас есть эталонный интеграл $$I = \int_{0}^\pi \sin(x) dx = 2$$

Посчитаем интеграл $I_1$ с шагом $h$. Остаток $R_1 = |I - I_1|$ Затем уменьшим шаг в $k$ раз и вычислим $I_2$, $R_2 = |I - I_2|$
Далее вычисляем

In [38]:
I = 2 # эталонный интеграл
interval = (0, np.pi)
I1 = integration(h, interval, 100)
I2 = integration(h, interval, 1000)

R1 = np.abs(I1 - I)
R2 = np.abs(I2 - I)
R = (R1 / R2) / 10 ** 4 # R1 = h ** 4 R2 = h ** 4 / 10**4 order like 10**4 
print(f"{R:.10f}")

2.000000000001081
1.0014221446


теперь когда результат интегрирования неизвестен будем сравнивать I1 и I2 
$$ I_1 = I + C \cdot (R_1)^p $$
$$ I_2 = I + C \cdot (R_2)^p $$
$$ I_3 = I + C \cdot (R_3)^p $$

$$ I_2 - I_3 =  C \cdot (R_2 ^ p - R_3 ^ p) $$
$$ I_1 - I_3 = C \cdot (R_1 ^ p - R_3 ^ p) $$
$$ \frac {I_1 - I_3} {I_2 - I_3} = \frac {R_1 ^ p - R_3 ^ p} {R_2 ^ p - R_3 ^ p} $$

Неизвестные C, p, I




In [37]:

interval = (0, 1)

I1 = integration(f, interval, 10)
I2 = integration(f, interval, 100)
I3 = integration(f, interval, 1000)
T1 = np.abs(I1 - I3)
T2 = np.abs(I2 - I3)



R = (T1 / T2)
print(f"{R:.10f}")



1.2368421053
